In [1]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import ta
import scipy.stats as stats


class Forex(object):
    def __init__(self, ticker: str, time_frame: str):
        self.ticker = ticker
        self.time_frame = time_frame

    def load_data(ticker: str, time_frame: str) -> None: # Load dataset.
        params_dic = {
            'host': 'localhost',
            'database': '/home/felipe/Desktop/python/projects/algorithmic-trading/data/forex.db'
        }
        with sqlite3.connect(f"{params_dic['database']}") as connection:
            dataset = pd.read_sql_query("SELECT * FROM "+ticker.lower()+"_"+time_frame, connection)
            return dataset

class FeatureEngineering(Forex):
    def __init__(self, ticker: str, time_frame: str):
        super().__init__(ticker, time_frame)

    def load_features(self): # Load features
        dataset = Forex.load_data(self.ticker, self.time_frame)
        dataset.index = dataset['date']
        CCI = ta.trend.CCIIndicator(high=dataset['high'], low=dataset['low'], close=dataset['close'], window=21) # Initialize CCI
        WMA = ta.trend.WMAIndicator(close=dataset['close'], window=21, fillna=False) # Initialize VMA
        RSI = ta.momentum.RSIIndicator(close=dataset['close'], window=21, fillna=False) # Initialize RSI
        STOCH = ta.momentum.StochasticOscillator(close=dataset['close'], high=dataset['high'], low=dataset['low'], window=21, smooth_window=3, fillna=False)
        DonchianChannel = ta.volatility.DonchianChannel(high=dataset['high'], low=dataset['low'], close=dataset['close'], window=21, fillna=False)
        dataset['EMA8'] = dataset['close'].ewm(span=8, adjust=False).mean()
        dataset['EMA21'] = dataset['close'].ewm(span=21, adjust=False).mean()
        dataset['STOCH'] = STOCH.stoch() # (STOCH) - Momentum
        dataset['RSI'] = RSI.rsi() # (RSI) - Momentum
        dataset['CCI'] = CCI.cci() # (CCI) - Trend
        dataset['WMA'] = WMA.wma() # (VMA) - Trend
        dataset['Donchian'] = DonchianChannel.donchian_channel_mband() # DonchianChannel - Volatility
        dataset['date'] = pd.to_datetime(dataset['date'])
        dataset['weekday'] = datetime.datetime.today().weekday()
        dataset['normalize'] = dataset['close'] / dataset['close'].iloc[0]
        dataset['date'] = pd.to_datetime(dataset['date'])
        dataset['day'] = dataset['date'].dt.day
        dataset['day_of_year'] = dataset['date'].dt.dayofyear 
        dataset['weekday'] = dataset['date'].apply(lambda x: x.weekday()) # Monday = 0 / Friday = 4	
        dataset['month'] = dataset['date'].dt.month
        dataset['min_21'] = dataset['close'].rolling(window=21).min()
        dataset['max_21'] = dataset['close'].rolling(window=21).max()
        dataset['std_21'] = dataset['close'].rolling(window=21).std()
        dataset['sma_21'] = dataset['close'].rolling(window=21).mean()
        dataset['ema_21'] = dataset['close'].ewm(com=21, min_periods=21).mean()
        dataset['zscore'] = stats.zscore(dataset['close'])
        for lag in range(1, 4): 
            dataset[f'daily_return_{lag}'] = dataset['close'].pct_change(lag)
        for i in range(1, 4):
            dataset[f'difference_{i}'] = dataset['close'].diff(i)
        for i in range(1, 4):
            dataset[f'daily change%_{i}'] = ((dataset['close'] / dataset['close'].shift(i)) * 100 - 100)
        dataset['positive/negative'] = np.where(dataset['daily_return_1'] > 0, 1, 0)
        return pd.DataFrame(dataset.dropna())

currency_pair = ['AUDCAD']
#currency_pair = ['AUDCAD', 'AUDNZD', 'AUDUSD', 'CADCHF', 'EURAUD', 'EURBRL', 'EURCAD', 'EURCHF', 'EURHUF', 'EURGBP', 'EURJPY', 'EURNZD', 'EURUSD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 'GBPUSD', 'GBPNZD', 'USDCAD', 'USDCHF', 'USDJPY', 'USDMXN', 'USDSGD', 'USDTRY', 'NZDCAD', 'NZDUSD', 'NZDJPY']
for currency in currency_pair:
    #time_frames = ['1d', '1wk', '1mo', '3mo'] # Intervals: “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”
    time_frames = ['1d']
    for time_frame in time_frames:
        dataset = FeatureEngineering(currency, time_frame).load_features()

print(dataset.info())
dataset.tail()

<class 'pandas.core.frame.DataFrame'>
Index: 4767 entries, 2003-12-29 00:00:00 to 2022-04-16 00:00:00
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               4767 non-null   datetime64[ns]
 1   open               4767 non-null   float64       
 2   high               4767 non-null   float64       
 3   low                4767 non-null   float64       
 4   close              4767 non-null   float64       
 5   EMA8               4767 non-null   float64       
 6   EMA21              4767 non-null   float64       
 7   STOCH              4767 non-null   float64       
 8   RSI                4767 non-null   float64       
 9   CCI                4767 non-null   float64       
 10  WMA                4767 non-null   float64       
 11  Donchian           4767 non-null   float64       
 12  weekday            4767 non-null   int64         
 13  normalize          4767 non-null   

,date,open,high,low,close,EMA8,EMA21,STOCH,RSI,CCI,...,daily_return_1,daily_return_2,daily_return_3,difference_1,difference_2,difference_3,daily change%_1,daily change%_2,daily change%_3,positive/negative
date,,,,,,,,,,,,,,,,,,,,,
2022-04-12 00:00:00,2022-04-12,0.937261,0.944476,0.936295,0.937261,0.939318,0.936473,58.585451,54.241774,40.568193,...,0.000739,-0.004450,-0.005917,0.000692,-0.004189,-0.005579,0.073888,-0.444953,-0.591722,1
2022-04-13 00:00:00,2022-04-13,0.941400,0.942997,0.934900,0.941400,0.939781,0.936921,70.606952,56.640289,42.745676,...,0.004416,0.005158,-0.000053,0.004139,0.004831,-0.000050,0.441607,0.515821,-0.005312,1
2022-04-14 00:00:00,2022-04-14,0.936320,0.937684,0.932300,0.936320,0.939012,0.936866,55.852434,53.056306,-41.404092,...,-0.005396,-0.001004,-0.000266,-0.005080,-0.000941,-0.000249,-0.539620,-0.100397,-0.026583,0
2022-04-15 00:00:00,2022-04-15,0.934089,0.934770,0.931900,0.934089,0.937918,0.936614,22.939861,51.552078,-111.764860,...,-0.002383,-0.007766,-0.003384,-0.002231,-0.007311,-0.003172,-0.238273,-0.776608,-0.338431,0
2022-04-16 00:00:00,2022-04-16,0.931760,0.931760,0.931760,0.931760,0.936549,0.936172,12.643708,49.998299,-151.662858,...,-0.002493,-0.004870,-0.010240,-0.002329,-0.004560,-0.009640,-0.249333,-0.487012,-1.024005,0
